# Preprocessing Data

Data preprocessing adalah proses yang mengubah data mentah ke dalam bentuk yang lebih mudah dipahami. Proses ini penting dilakukan karena data mentah sering kali tidak memiliki format yang teratur. Selain itu, data mining juga tidak dapat memproses data mentah, sehingga proses ini sangat penting dilakukan untuk mempermudah proses berikutnya, yakni analisis data.


## Import Library

In [76]:
import pandas as pd
import numpy as np
import string
import re #regrex libray
import nltk
import swifter
import Sastrawi
import networkx as nx

from nltk.tokenize import word_tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

## Load Dataset

In [77]:
data_berita = pd.read_csv('ptamanajemen.csv')
data_berita.head()

,abstrak
0,ABSTRAK\r\...
1,ABSTRACT...
2,"Electronic journals, also known as ejournals, ..."


## Case Folding

In [78]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
data_berita['abstrak'] = data_berita['abstrak'].str.lower()


print('Case Folding Result : \n')
print(data_berita['abstrak'].head(20))
print('\n\n\n')

Case Folding Result : 

0                                        abstrak\r\...
1                                          abstract...
2    electronic journals, also known as ejournals, ...
Name: abstrak, dtype: object






## Tokenizing

In [79]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
nltk.download('punkt')
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
data_berita['abstrak'] = data_berita['abstrak'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

data_berita['abstrak'] = data_berita['abstrak'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

data_berita['abstrak'] = data_berita['abstrak'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

data_berita['abstrak'] = data_berita['abstrak'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

data_berita['abstrak'] = data_berita['abstrak'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

data_berita['abstrak'] = data_berita['abstrak'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

data_berita['abstrak'] = data_berita['abstrak'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(data_berita['abstrak'].head(20))
print('\n\n\n')

Tokenizing Result : 

0    [abstrak, satiyah, pengaruh, faktorfaktor, pel...
1    [abstract, in, an, effort, to, increase, labor...
2    [electronic, journals, also, known, as, ejourn...
Name: abstrak, dtype: object




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Fatin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Menghitung Frekuensi Distribusi Token

In [80]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

data_berita['abstrak'] = data_berita['abstrak'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
print(data_berita['abstrak'].head(20).apply(lambda x : x.most_common()))

Frequency Tokens : 

0    [(dan, 20), (dengan, 13), (produktivitas, 9), ...
1    [(the, 25), (of, 21), (and, 18), (to, 12), (pr...
2    [(are, 4), (they, 4), (electronic, 3), (journa...
Name: abstrak, dtype: object


## Filtering (Stopword Removal)

In [81]:
nltk.download('stopwords')

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("ptamanajemen.csv", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

data_berita['abstrak'] = data_berita['abstrak'].apply(stopwords_removal) 


print(data_berita['abstrak'].head(20))

0    [produktivitas, kerja, pelatihan, pengembangan...
1    [the, of, and, to, productivity, is, training,...
2    [are, they, electronic, journals, ejournals, t...
Name: abstrak, dtype: object
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Fatin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Normalization

In [82]:
normalizad_word = pd.read_csv("ptamanajemen.csv")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[0] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

data_berita['abstrak'] = data_berita['abstrak'].apply(normalized_term)

data_berita['abstrak'].head()

0    [produktivitas, kerja, pelatihan, pengembangan...
1    [the, of, and, to, productivity, is, training,...
2    [are, they, electronic, journals, ejournals, t...
Name: abstrak, dtype: object

## Stemming

In [83]:

# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in data_berita['abstrak']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    i=0
    if i<10:
        term_dict[term] = stemmed_wrapper(term)
        #print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

data_berita['abstrak'] = data_berita['abstrak'].swifter.apply(get_stemmed_term)
print(data_berita['abstrak'])

292
------------------------


Pandas Apply:   0%|          | 0/3 [00:00<?, ?it/s]{'produktivitas': 'produktivitas', 'kerja': 'kerja', 'pelatihan': 'latih', 'pengembangan': 'kembang', 'pegawai': 'pegawai', 'dinas': 'dinas', 'kelautan': 'laut', 'perikanan': 'ikan', 'bangkalan': 'bangkal', 'pengaruh': 'pengaruh', 'faktorfaktor': 'faktorfaktor', 'penelitian': 'teliti', 'meningkat': 'tingkat', 'instansi': 'instansi', 'hubungan': 'hubung', 'metode': 'metode', 'seleksi': 'seleksi', 'variabel': 'variabel', 'responden': 'responden', 'dianalisis': 'analis', 'sampling': 'sampling', 'analisis': 'analisis', 'jabatan': 'jabat', 'motivasi': 'motivasi', 'peserta': 'serta', 'kabupaten': 'kabupaten', 'terbukti': 'bukti', 'nilai': 'nilai', 'satiyah': 'satiyah', 'dibawah': 'bawah', 'bimbingan': 'bimbing', 'drahjsanugrahini': 'drahjsanugrahini', 'irawatimm': 'irawatimm', 'helmi': 'helm', 'buyung': 'buyung', 'auliasstsemmt': 'auliasstsemmt', 'upaya': 'upaya', 'meningkatkan': 'tingkat', 'mudah': 'mudah', 's

AttributeError: 'Series' object has no attribute '_is_builtin_func'

## Simpan Data ke CSV

In [84]:
data_berita.to_csv("TextPreprocessing.csv")

## Prepare Data

In [85]:
import pandas as pd 
import numpy as np

data_berita = pd.read_csv("TextPreprocessing.csv", usecols=["abstrak"])
data_berita.columns = ["abstrak"]

data_berita.head(20)

,abstrak
0,"['produktivitas', 'kerja', 'pelatihan', 'penge..."
1,"['the', 'of', 'and', 'to', 'productivity', 'is..."
2,"['are', 'they', 'electronic', 'journals', 'ejo..."


## Term Frekuensi

In [86]:
from sklearn.feature_extraction.text import CountVectorizer

a=len(document)
document = data_berita['abstrak']

# Create a Vectorizer Object
vectorizer = CountVectorizer()

vectorizer.fit(document)

# Printing the identified Unique words along with their indices
print("Vocabulary: ", vectorizer.vocabulary_)

# Encode the Document
vector = vectorizer.transform(document)

# Summarizing the Encoded Texts
print("Encoded Document is:")
print(vector.toarray())

Vocabulary:  {'produktivitas': 208, 'kerja': 127, 'pelatihan': 185, 'pengembangan': 191, 'pegawai': 184, 'dinas': 59, 'kelautan': 125, 'perikanan': 197, 'bangkalan': 31, 'pengaruh': 190, 'faktorfaktor': 82, 'penelitian': 188, 'meningkat': 153, 'instansi': 112, 'hubungan': 100, 'metode': 158, 'seleksi': 234, 'variabel': 280, 'responden': 222, 'dianalisis': 54, 'sampling': 227, 'analisis': 15, 'jabatan': 120, 'motivasi': 159, 'peserta': 199, 'kabupaten': 123, 'terbukti': 255, 'nilai': 168, 'satiyah': 228, 'dibawah': 55, 'bimbingan': 38, 'drahjsanugrahini': 65, 'irawatimm': 117, 'helmi': 96, 'buyung': 39, 'auliasstsemmt': 30, 'upaya': 274, 'meningkatkan': 154, 'mudah': 162, 'salah': 224, 'usaha': 275, 'menerapkan': 150, 'program': 210, 'sumber': 252, 'daya': 48, 'manusia': 144, 'sdm': 230, 'dilaksanakan': 58, 'tercapai': 256, 'kemampuan': 126, 'efektif': 67, 'efisien': 72, 'pengembnagan': 192, 'diharapkan': 57, 'menyesuaikan': 155, 'kebutuhankebutuhan': 124, 'sikap': 237, 'tingkah': 266, 

In [87]:
a = vectorizer.get_feature_names()

## TF-IDF

In [88]:
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
tf = tfidf.fit_transform(vectorizer.fit_transform(document)).toarray()

In [89]:
dfb = pd.DataFrame(data=tf,index=list(range(1, len(tf[:,1])+1, )),columns=[a])
# dfb

## Convert File to CSV

In [90]:
dfb.to_csv("TF-IDF.csv")